# Answers to lecture

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By

import time
import math

## Set up 

Get `webdriver`, the dates, max rows visible, etc., all set up here.

In [2]:
# initiate webdriver
driver = webdriver.Chrome()

In [3]:
# "get" the webpage
driver.get('https://publicrecords.alameda.courts.ca.gov/CalendarSearch/')

In [4]:
hearing_date_from = driver.find_element(By.ID, 'FeaturedContent_txtFromdt')
hearing_date_to   = driver.find_element(By.ID, 'FeaturedContent_txtTodt')

In [5]:
hearing_date_from.send_keys('12/06/2021')
hearing_date_to.send_keys('12/06/2021')

In [6]:
submit_button = driver.find_element(By.ID, 'FeaturedContent_btFind')
submit_button.click()
time.sleep(5)

In [7]:
displayed_rows_dropdown = Select(driver.find_element(By.NAME, 'ctl00$MainContent$gvResult$ctl13$ctl13'))
displayed_rows_dropdown.select_by_visible_text('50')

In [8]:
records_count_container = driver.find_element(By.ID, 'MainContent_lbCnt')
records_count = records_count_container.text.split()
records_count = records_count[len(records_count) - 1]
records_count = int(records_count)
records_count

1395

In [9]:
pages_to_check = math.ceil(records_count/50)
pages_to_check

28

## Create an empty dataframe to store all the data

In [10]:
hearings = pd.DataFrame(
    columns=[
        'Serial No.',
        'Name',
        'Case #',
        'PFN',
        'CEN',
        'Dept#',
        'Hearing Date',
        'Hearing Time',
        'Hearing Type',
        'Case Type',
        'Defense Atty',
        'DA'
    ])

## Loop through all the pages

### Below I wrote a function `scrape_table()` that I can call from the loop.

In [11]:
def scrape_table(table):
    page_data = []
    rows = table.find_all('tr')
    for i, row in enumerate(rows):
        # we can skip the first row because that's the header row
        if (i > 0) and (i <= 50):
            cells = row.find_all('td')
            page_data.append({
                'Serial No.' : cells[0].text.strip(),
                'Name' : cells[1].text.strip(), 
                'Case #' : cells[2].text.strip(), 
                'PFN' : cells[3].text.strip(), 
                'CEN' : cells[4].text.strip(), 
                'Dept#' : cells[5].text.strip(), 
                'Hearing Date' : cells[6].text.strip(), 
                'Hearing Time' : cells[7].text.strip(), 
                'Hearing Type' : cells[8].text.strip(), 
                'Case Type' : cells[9].text.strip(), 
                'Defense Atty' : cells[10].text.strip(), 
                'DA' : cells[11].text.strip()
            })
    return pd.DataFrame(page_data)

## Run the actual loop

In [12]:
for n in tqdm(range(pages_to_check)):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find(id='MainContent_gvResult')
    
    # scrape the page and create a dataframe
    page_df = scrape_table(table)
    
    # append the page dataframe to the main `hearings` dataframe
    hearings = hearings.append(page_df).reset_index(drop=True)
    
    # click on the next button and pause for 5 seconds
    next_button = driver.find_element(By.LINK_TEXT, '>')
    next_button.click()
    time.sleep(5)

  0%|          | 0/28 [00:00<?, ?it/s]

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":">"}
  (Session info: chrome=96.0.4664.55)
Stacktrace:
0   chromedriver                        0x000000010277f269 __gxx_personality_v0 + 582729
1   chromedriver                        0x000000010270ac33 __gxx_personality_v0 + 106003
2   chromedriver                        0x00000001022c7e28 chromedriver + 171560
3   chromedriver                        0x00000001022fd3d2 chromedriver + 390098
4   chromedriver                        0x00000001022fd591 chromedriver + 390545
5   chromedriver                        0x000000010232f6b4 chromedriver + 595636
6   chromedriver                        0x000000010231a9fd chromedriver + 510461
7   chromedriver                        0x000000010232d462 chromedriver + 586850
8   chromedriver                        0x000000010231ac23 chromedriver + 511011
9   chromedriver                        0x00000001022f075e chromedriver + 337758
10  chrome

## View the resulting dataframe

In [13]:
hearings

,Serial No.,Name,Case #,PFN,CEN,Dept#,Hearing Date,Hearing Time,Hearing Type,Case Type,Defense Atty,DA
0,1,"FOSTER, JIMMIE LEE",167627,AYO984,1340030,,12/06/2021,9:00AM,Hearing,Felony,,
1,2,"THOMAS, MARKEL",410407,BGN192,8324445,,12/06/2021,9:00AM,Hearing,Misdemeanor/Infractions,,
2,3,"MOORE, MARIA",18-CR-019696B,BMD439,8418630,002,12/06/2021,8:30AM,Further Jury Trial,Felony,Darryl Antonio Stallworth,Alexander Hernandez
3,4,"SALVANT, MARVEL",18-CR-019696A,BMD440,8418631,002,12/06/2021,8:30AM,Further Jury Trial,Felony,Christina Marie Moore,Alexander Hernandez
4,5,"STANDLEY, CURTIS RICHARD",17-CR-027648,BGT167,7831868,003,12/06/2021,8:30AM,Further Jury Trial,Felony,Anne Catherine Beles,Angelina Lisa Clay
...,...,...,...,...,...,...,...,...,...,...,...,...
1354,1393,"SMILEY, JASMON DANTE",21-CR-003317,DTF918,1368625,713,12/06/2021,8:30AM,Preliminary Hearing,Felony,Alexandria Grayson Blythe,Brooke Danielle Perkins
1355,1394,"SMILEY, JASMON DANTE",21-CR-003317,BDC095,1368625,713,12/06/2021,8:30AM,Preliminary Hearing,Felony,Alexandria Grayson Blythe,Brooke Danielle Perkins
1356,1395,"WELLS, DAVID MICHAEL",21-CR-012227,ASE868,1388635,713,12/06/2021,8:30AM,Preliminary Hearing,Felony,Alexandria Grayson Blythe,Brooke Danielle Perkins
1357,<<First<...19202122232425262728Rows:\n10\n20\n50,<<First,<,...,19,20,21,22,23,24,25,26


## Clean up data

In [15]:
hearings['Hearing Type'].unique()

array(['Hearing', 'Further Jury Trial',
       'Hearing After Revocation of Probation', 'Bail Hearing',
       'Bail Bond Motion', 'Motion pursuant to 995PC', '8', '9',
       'Sentence After Revocation', 'Report and Sentence',
       'Set Hearing After Revocation of Probation',
       'Parole Revocation Hearing',
       'Post Release Community Supervision Hearing', 'Pretrial Hearing',
       'Motion to Dismiss', 'Arraignment', 'Jury Trial',
       'Readiness Hearing', 'For Review',
       'District Attorney Petition to Revoke Probation', 'Plea Hearing',
       'Disposition Hearing', 'Recall Bench Warrant',
       'Drug Court Status Hearing', 'Diversion Hearing',
       'Hearing on Petitions under Prop 47',
       'Progress Report Early Intervention Court',
       'Attorney and Plea Hearing', 'Progress Report',
       'Execution of Sentence', 'Issue Bench Warrant',
       '977 - Misdemeanor Disposition', 'Preliminary Hearing', 'Set',
       'Summary Judgment Hearing', 'Resentencing/mod

In [16]:
to_clean = ['8', '9', '23', '24']
hearings[hearings['Hearing Type'].isin(to_clean)]

,Serial No.,Name,Case #,PFN,CEN,Dept#,Hearing Date,Hearing Time,Hearing Type,Case Type,Defense Atty,DA
10,12345678910...>Last>>Rows:\n10\n20\n50,1,2,3,4,5,6,7,8,9,10,...
11,1,2,3,4,5,6,7,8,9,10,...,>
1357,<<First<...19202122232425262728Rows:\n10\n20\n50,<<First,<,...,19,20,21,22,23,24,25,26
1358,<<First,<,...,19,20,21,22,23,24,25,26,27


In [17]:
hearings = hearings[~hearings['Hearing Type'].isin(to_clean)].reset_index(drop=True)

In [19]:
hearings.tail()

,Serial No.,Name,Case #,PFN,CEN,Dept#,Hearing Date,Hearing Time,Hearing Type,Case Type,Defense Atty,DA
1350,1391,"PRIETO, STEVE",21-CR-009297,BMZ282,1377159,713,12/06/2021,8:30AM,Preliminary Hearing and Motion to Suppress,Felony,Laura R Bull,Brooke Danielle Perkins
1351,1392,"RUIZ, TRINA LYNN",21-CR-011772,AYV628,1387766,713,12/06/2021,8:30AM,Preliminary Hearing,Felony,Robbi Abrams Cook,Brooke Danielle Perkins
1352,1393,"SMILEY, JASMON DANTE",21-CR-003317,DTF918,1368625,713,12/06/2021,8:30AM,Preliminary Hearing,Felony,Alexandria Grayson Blythe,Brooke Danielle Perkins
1353,1394,"SMILEY, JASMON DANTE",21-CR-003317,BDC095,1368625,713,12/06/2021,8:30AM,Preliminary Hearing,Felony,Alexandria Grayson Blythe,Brooke Danielle Perkins
1354,1395,"WELLS, DAVID MICHAEL",21-CR-012227,ASE868,1388635,713,12/06/2021,8:30AM,Preliminary Hearing,Felony,Alexandria Grayson Blythe,Brooke Danielle Perkins


In [22]:
hearings['Serial No.'] = pd.to_numeric(hearings['Serial No.'])
hearings['Dept#'] = pd.to_numeric(hearings['Dept#'])

In [23]:
hearings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1355 entries, 0 to 1354
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Serial No.    1355 non-null   int64  
 1   Name          1355 non-null   object 
 2   Case #        1355 non-null   object 
 3   PFN           1355 non-null   object 
 4   CEN           1355 non-null   object 
 5   Dept#         1353 non-null   float64
 6   Hearing Date  1355 non-null   object 
 7   Hearing Time  1355 non-null   object 
 8   Hearing Type  1355 non-null   object 
 9   Case Type     1355 non-null   object 
 10  Defense Atty  1355 non-null   object 
 11  DA            1355 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 127.2+ KB


In [24]:
hearings

,Serial No.,Name,Case #,PFN,CEN,Dept#,Hearing Date,Hearing Time,Hearing Type,Case Type,Defense Atty,DA
0,1,"FOSTER, JIMMIE LEE",167627,AYO984,1340030,NaN,12/06/2021,9:00AM,Hearing,Felony,,
1,2,"THOMAS, MARKEL",410407,BGN192,8324445,NaN,12/06/2021,9:00AM,Hearing,Misdemeanor/Infractions,,
2,3,"MOORE, MARIA",18-CR-019696B,BMD439,8418630,2.0,12/06/2021,8:30AM,Further Jury Trial,Felony,Darryl Antonio Stallworth,Alexander Hernandez
3,4,"SALVANT, MARVEL",18-CR-019696A,BMD440,8418631,2.0,12/06/2021,8:30AM,Further Jury Trial,Felony,Christina Marie Moore,Alexander Hernandez
4,5,"STANDLEY, CURTIS RICHARD",17-CR-027648,BGT167,7831868,3.0,12/06/2021,8:30AM,Further Jury Trial,Felony,Anne Catherine Beles,Angelina Lisa Clay
...,...,...,...,...,...,...,...,...,...,...,...,...
1350,1391,"PRIETO, STEVE",21-CR-009297,BMZ282,1377159,713.0,12/06/2021,8:30AM,Preliminary Hearing and Motion to Suppress,Felony,Laura R Bull,Brooke Danielle Perkins
1351,1392,"RUIZ, TRINA LYNN",21-CR-011772,AYV628,1387766,713.0,12/06/2021,8:30AM,Preliminary Hearing,Felony,Robbi Abrams Cook,Brooke Danielle Perkins
1352,1393,"SMILEY, JASMON DANTE",21-CR-003317,DTF918,1368625,713.0,12/06/2021,8:30AM,Preliminary Hearing,Felony,Alexandria Grayson Blythe,Brooke Danielle Perkins
1353,1394,"SMILEY, JASMON DANTE",21-CR-003317,BDC095,1368625,713.0,12/06/2021,8:30AM,Preliminary Hearing,Felony,Alexandria Grayson Blythe,Brooke Danielle Perkins


## Export the dataframe

In [ ]:
hearings.to_csv('hearings.csv', index=False)

## Explore the dataframe

In [ ]:
hearing_types = hearings['Hearing Type'].unique()

In [ ]:
hearing_types.sort()
hearing_types

In [ ]:
hearings[hearings['Hearing Type'] == 'Drug Court Status Hearing']

In [ ]:
Possible story idea: Drug courts?

https://www.dhcs.ca.gov/individuals/Pages/Drug-Courts-Overview.aspx

## Close the automated browser

In [ ]:
driver.close()